In [1]:
import numpy as np
import random 
import math

Representation of the chromosome: 
Array of bits where each element denotes whether an item is included (1) or not (0).
E.g. [0,1,0] => Item 1 and 3 are not included, item 2 is included.

### 0. Data

In [2]:
ITERATION_LIMIT = 3
POPULATION_SIZE = 20
CROSSOVER_PROB = 0.8
MUTATION_PROB = 0.1
MAX_WEIGHT = 53

weights = [3, 12, 8, 11, 10, 7, 6, 2, 14, 2]
values = [5, 9, 1, 14, 8, 12, 5, 6, 3, 7]

### 1. Initial population

In [3]:
def initialize_population(population_size, item_amount):
    chromosomes = []
    for i in range(population_size):
        chromosome = []
        for k in range(item_amount):
            k = random.randint(0, 1)
            chromosome.append(k)    
        chromosomes.append(chromosome)
        
    print(">>> Initial population:")
    print(np.matrix(chromosomes))
    print("")
    return chromosomes

### 2. Adaptation function

In [4]:
def calculate_fitness(chromosome):
    sum_w = 0
    sum_v = 0

    # get weights and values
    for index, i in enumerate(chromosome):
        if i == 0:
            continue
        else:
            sum_w += weights[index]
            sum_v += values[index]
    return sum_w, sum_v

Handle overloaded knapsacks by inverting each bit 1 of the chromosome and recalculating untill chromosome is not overloaded anymore

In [5]:
def calculate_fitness_non_overloaded(chromosome):
    weight, value = calculate_fitness(chromosome)
    while weight > MAX_WEIGHT:
        invert_first_one_bit(chromosome)
        weight, value = calculate_fitness(chromosome)
        
    #print("Processed chromosome:", chromosome)
    return value
    
def invert_first_one_bit(chromosome):
    for index, i in enumerate(chromosome):
        if i == 0:
            continue
        else:
            #print("Invert 1 at index", index)
            chromosome[index] = 0
            return chromosome

### 3. Chromosome selection - roulette wheel

Create a fitness list of chromosomes in a population

In [6]:
def get_fitness_list(population):
    fitness_list = []
    for chromosome in population:
        fitness_list.append(calculate_fitness_non_overloaded(chromosome))
    
    return fitness_list

Create a probability list (imaginary proportion of the wheel)

In [7]:
def get_probability_list(population):
    fitness_list = get_fitness_list(population)
    total_fitness = float(sum(fitness_list))
    relative_fitness = [f/total_fitness for f in fitness_list]
    probability_list = [sum(relative_fitness[:i+1]) for i in range(len(relative_fitness))]
    return probability_list

Use roulette wheel to select chromosomes from the population and form a parental pool with a given size number

In [8]:
def roulette(population, pool_size):
    parental_pool = []
    
    probability_list = get_probability_list(population)
    for n in range(pool_size):
        r = random.random()
        for (i, individual) in enumerate(population):
            if r <= probability_list[i]:
                parental_pool.append(list(individual))
                break
    print(">>> Parental pool by roulette wheel:")
    print(np.matrix(parental_pool))
    return parental_pool

### 4. Generic operations

#### Crossover

In [9]:
def crossover(chromosome1, chromosome2, probability):
    if random.uniform(0, 1) < probability:
        print("Input: ", chromosome1, chromosome2)
        locus = random.randint(1, len(chromosome1)-1)
    
        temp1 = chromosome1[locus:]
        temp2 = chromosome2[locus:]
        chromosome1 = chromosome1[:locus]
        chromosome2 = chromosome2[:locus]
        chromosome1.extend(temp2)
        chromosome2.extend(temp1)
        print("Output:", chromosome1, chromosome2)
        
    return chromosome1, chromosome2

#### Mutation

In [10]:
def mutate(chromosome, probability):
    print("Input: ", chromosome)
    for index in range(len(chromosome)):
        if random.uniform(0, 1) < probability:
            # flip 0 with 1 and vice versa
            if chromosome[index] == 1:
                chromosome[index] = 0
            else: 
                chromosome[index] = 1
    print("Output:", chromosome)
    return chromosome

### 5. Termination condition

In [11]:
def termination_condition_meet(iteration_count):
    # TODO: The population converages when either X% (normally > 90%) of the chromosomes 
    # in the population have the same fitness value
    
    # number of generations exeeds a limit number
    return iteration_count >= ITERATION_LIMIT

### Algorithm flow

In [14]:
iteration_count = 0
population = initialize_population(POPULATION_SIZE, len(values))

while not termination_condition_meet(iteration_count):
    print("ITERATION", iteration_count + 1)
    
    population = roulette(population, POPULATION_SIZE)
    
    print(">>> Crossover chromosomes")
    # when crossover, exclude the last chromosome if population size is odd
    for index in range(0, math.floor(len(population) / 2) * 2, 2):
        crossover(population[index], population[index + 1], CROSSOVER_PROB)
    
    print(">>> Imutate chromosomes")
    for index in range(len(population)):
        mutate(population[index], MUTATION_PROB)

    print(">>> Fitness values after iteration", iteration_count + 1)
    print(get_fitness_list(population))
    print("")
    iteration_count += 1

# finish iterating, get the best chromosome
print("\n>>> CONCLUSION")
fitness_list = get_fitness_list(population)
print(">>> Best chromosome:")
best_chromosome = population[np.argmax(fitness_list)]
print(best_chromosome)
for index in range(len(best_chromosome)):
    if(best_chromosome[index] == 1):
        print("Item", index + 1, "| weight:", weights[index], "| value:", values[index],  "|")

total_weight, total_value = calculate_fitness(best_chromosome)
print(">>> Total weight:")
print(total_weight)
print(">>> Total value:")
print(total_value)

>>> Initial population:
[[1 1 0 0 0 1 1 0 0 0]
 [0 0 1 1 0 0 0 1 0 0]
 [0 1 0 0 0 1 1 0 0 0]
 [1 0 0 0 0 1 1 1 0 0]
 [0 0 1 1 1 0 1 1 1 1]
 [0 0 0 1 0 0 0 1 0 1]
 [1 0 0 0 1 1 0 0 1 0]
 [0 1 0 0 1 1 0 0 1 1]
 [0 0 1 1 0 0 0 0 1 0]
 [1 0 0 1 0 1 1 0 1 1]
 [0 0 1 1 0 1 0 0 0 1]
 [1 0 1 1 0 1 0 0 0 1]
 [0 1 0 0 0 0 1 1 0 1]
 [0 1 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 0 1 0 1]
 [0 0 0 1 0 1 1 1 1 0]
 [0 0 0 1 0 0 0 0 1 0]
 [0 1 0 1 0 1 0 1 1 1]
 [0 1 0 0 0 1 1 1 0 0]
 [1 1 1 0 0 1 1 1 0 0]]

ITERATION 1
>>> Parental pool by roulette wheel:
[[0 0 0 1 0 0 0 1 0 1]
 [0 1 0 0 0 1 1 0 0 0]
 [0 0 0 1 0 0 0 0 1 0]
 [1 0 1 1 0 1 0 0 0 1]
 [1 0 0 0 1 0 0 1 0 1]
 [0 0 1 1 0 1 0 0 0 1]
 [1 0 1 1 0 1 0 0 0 1]
 [0 1 0 0 0 1 1 0 0 0]
 [0 1 0 0 0 1 1 1 0 0]
 [0 0 0 1 0 1 1 1 1 0]
 [1 0 1 1 0 1 0 0 0 1]
 [0 0 1 1 0 1 0 0 0 1]
 [1 0 0 1 0 1 1 0 1 1]
 [0 1 0 0 1 1 0 0 1 1]
 [0 0 0 1 0 1 1 1 1 0]
 [0 1 0 0 0 0 1 1 0 1]
 [1 1 1 0 0 1 1 1 0 0]
 [0 0 0 1 0 0 0 1 0 1]
 [0 1 0 1 0 0 0 0 1 0]
 [1 0 1 1 0 1 0 0 0 1]]
>>> 